<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Darvas_Box_Dual_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed backtesting openpyxl
!pip install tradingview-screener==2.5.0

# Gerekli kütüphaneler
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import openpyxl

# Uyarıları yok sayma
warnings.simplefilter(action='ignore', category=FutureWarning)

# Darvas Box stratejisini uygular
def darvas_box_strategy(data, bars_20=20, bars_50=50):
    df = data.copy()
    df['High_Max_20'] = df['High'].rolling(window=bars_20).max()
    df['Low_Min_20'] = df['Low'].rolling(window=bars_20).min()
    df['Darvas_Box_Up_20'] = df['Close'] > df['High_Max_20'].shift(1)
    df['Darvas_Box_Down_20'] = df['Close'] < df['Low_Min_20'].shift(1)
    df['High_Max_50'] = df['High'].rolling(window=bars_50).max()
    df['Low_Min_50'] = df['Low'].rolling(window=bars_50).min()
    df['Darvas_Box_Up_50'] = df['Close'] > df['High_Max_50'].shift(1)
    df['Darvas_Box_Down_50'] = df['Close'] < df['Low_Min_50'].shift(1)
    return df

# Kazanma oranını hesaplama fonksiyonu
def calculate_win_rate(df, buy_column, sell_column):
    buy_signals = df[df[buy_column] == True]
    sell_signals = df[df[sell_column] == True]

    win_count = 0
    total_count = min(len(buy_signals), len(sell_signals))

    for i in range(total_count):
        buy_price = buy_signals['Close'].iloc[i]
        sell_price = sell_signals['Close'].iloc[i]
        if sell_price > buy_price:
            win_count += 1

    if total_count > 0:
        return (win_count / total_count) * 100
    return 0

# TradingView API'ye bağlan
tv = TvDatafeed()

# BIST hisse senetlerini al
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Zaman dilimlerini tanımla
intervals = {
    '1': ('15 Dakika', Interval.in_15_minute),
    '2': ('30 Dakika', Interval.in_30_minute),
    '3': ('45 Dakika', Interval.in_45_minute),
    '4': ('1 Saat', Interval.in_1_hour),
    '5': ('2 Saat', Interval.in_2_hour),
    '6': ('4 Saat', Interval.in_4_hour),
    '7': ('Günlük', Interval.in_daily),
    '8': ('Haftalık', Interval.in_weekly),
    '9': ('Aylık', Interval.in_monthly)
}

# Kullanıcıdan zaman dilimi seçimini al
print("Lütfen bir zaman dilimi seçin:")
for key, (label, _) in intervals.items():
    print(f"{key}: {label}")
selected_interval = input("Seçiminizi yapın (1-9): ")

# Geçerli seçim kontrolü
if selected_interval not in intervals:
    print("Geçersiz seçim! Lütfen 1 ile 9 arasında bir değer girin.")
else:
    interval_name, interval_value = intervals[selected_interval]
    print(f"Seçilen zaman dilimi: {interval_name}")

    # Sonuçları saklamak için DataFrame'ler
    df_results = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Kazanma Oranı 20', 'Kazanma Oranı 50', 'Giriş Sinyali 20', 'Giriş Sinyali 50'])
    df_true_signals = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Kazanma Oranı 20', 'Kazanma Oranı 50', 'Giriş Sinyali 20', 'Giriş Sinyali 50'])

    # Hisseler üzerinde tarama
    for hisse in Hisseler:
        try:
            # Veri çek
            data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval_value, n_bars=500)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()

            # Darvas stratejisini uygula
            darvas_data = darvas_box_strategy(data)

            # Son fiyat ve sinyaller
            last_price = data['Close'].iloc[-1]
            buy_signal_20 = darvas_data['Darvas_Box_Up_20'].iloc[-1]
            buy_signal_50 = darvas_data['Darvas_Box_Up_50'].iloc[-1]

            # Kazanma oranlarını hesapla
            win_rate_20 = calculate_win_rate(darvas_data, 'Darvas_Box_Up_20', 'Darvas_Box_Down_20')
            win_rate_50 = calculate_win_rate(darvas_data, 'Darvas_Box_Up_50', 'Darvas_Box_Down_50')

            # Sonuçları ekle
            df_results.loc[len(df_results)] = [hisse, last_price, win_rate_20, win_rate_50, buy_signal_20, buy_signal_50]

            # True sinyalleri kontrol ve ekleme
            if buy_signal_20 or buy_signal_50:
                df_true_signals.loc[len(df_true_signals)] = [hisse, last_price, win_rate_20, win_rate_50, buy_signal_20, buy_signal_50]
                print(f"{hisse}, Son Fiyat: {last_price}, Kazanma Oranı 20: {win_rate_20:.2f}%, Kazanma Oranı 50: {win_rate_50:.2f}%, Giriş 20: {buy_signal_20}, Giriş 50: {buy_signal_50}")

        except Exception as e:
            print(f"Hata: {hisse} - {e}")

    # Sonuçları Excel'e kaydet
    excel_file = 'darvas_box_results.xlsx'
    with pd.ExcelWriter(excel_file) as writer:
        df_results.to_excel(writer, sheet_name='Tüm Sonuçlar', index=False)
        df_true_signals.to_excel(writer, sheet_name='True Sinyaller', index=False)

    print(f"Sonuçlar '{excel_file}' dosyasına kaydedildi.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-97ru7e1i
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-97ru7e1i
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.5/178.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=28c0510a01f5deec7bf2171b2678ccd39b45a5b9c1f84240b247891c4ed23449
  Stored in directory: /tmp/pip-ephem-wheel-cache-9zz1peh7/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00


Lütfen bir zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 7
Seçilen zaman dilimi: Günlük
AKCNS, Son Fiyat: 206.3, Kazanma Oranı 20: 33.33%, Kazanma Oranı 50: 100.00%, Giriş 20: True, Giriş 50: True
AKGRT, Son Fiyat: 8.23, Kazanma Oranı 20: 68.00%, Kazanma Oranı 50: 78.57%, Giriş 20: True, Giriş 50: True
ASELS, Son Fiyat: 85.15, Kazanma Oranı 20: 47.62%, Kazanma Oranı 50: 27.27%, Giriş 20: True, Giriş 50: True
ASTOR, Son Fiyat: 128.0, Kazanma Oranı 20: 42.11%, Kazanma Oranı 50: 30.43%, Giriş 20: True, Giriş 50: True
BINBN, Son Fiyat: 101.5, Kazanma Oranı 20: 0.00%, Kazanma Oranı 50: 0.00%, Giriş 20: True, Giriş 50: False
BSOKE, Son Fiyat: 12.83, Kazanma Oranı 20: 60.87%, Kazanma Oranı 50: 18.18%, Giriş 20: True, Giriş 50: False
CIMSA, Son Fiyat: 55.5, Kazanma Oranı 20: 25.93%, Kazanma Oranı 50: 54.55%, Giriş 20: True, Giriş 50: True
DCTTR, Son Fiyat: 41.5, Kazanma Oranı 20: 0.00%, Kazanma Or